# Brain Alignment: Comparing Models to Brains

**Quantifying how well artificial neural networks match biological neural activity**

## What You'll Learn

In this notebook, you'll master:
1. **Canonical Correlation Analysis (CCA)** - Finding shared representational spaces
2. **Representational Similarity Analysis (RSA)** - Comparing geometries without alignment
3. **Partial Least Squares (PLS)** - Predictive alignment with latent variables
4. **Statistical evaluation** - Noise ceilings, significance testing, confidence intervals
5. **Multi-layer alignment** - Finding which model layers match brain regions
6. **Practical workflows** - Complete analysis pipelines

## The Central Question

> **Do artificial neural networks process information like the brain?**

To answer this, we need rigorous methods to compare model representations with brain recordings (fMRI, EEG, MEG, neural recordings).

## Prerequisites

- Completed Notebooks 01-04
- Understanding of linear algebra
- Familiarity with neuroscience recordings (helpful but not required)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA as SklearnCCA, PLSRegression
from scipy.stats import spearmanr, pearsonr
from scipy.spatial.distance import pdist, squareform
from typing import Dict, List, Tuple
from tqdm.auto import tqdm

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Random seeds
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Note: Some alignment tools require sklearn
try:
    from neuros_mechint.alignment import CCA, RSA, PLS
    print("neuros-mechint alignment tools available")
except ImportError:
    print("Using sklearn implementations (neuros-mechint alignment optional)")

## Part 1: Understanding the Problem

### Why Compare Models to Brains?

**For Neuroscience:**
- Validate that models use brain-like computations
- Test hypotheses about neural coding
- Predict neural responses to new stimuli
- Understand information processing hierarchies

**For AI:**
- Guide architecture design toward biological plausibility
- Discover computational principles that work in nature
- Improve robustness and generalization
- Enable brain-computer interfaces

### The Challenge

Models and brains differ in many ways:
- **Different representations**: Model features vs neural activity patterns
- **Different dimensions**: 1000s of neurons vs 1000s of model units
- **Different scales**: ms (neurons) vs sec (fMRI)
- **Different noise**: Neural variability vs computational determinism

We need methods that can compare despite these differences!

### Three Complementary Approaches

1. **CCA**: Find linear transformations that maximize correlation
   - "What shared information do model and brain encode?"
   - Linear alignment required

2. **RSA**: Compare pairwise similarity structures
   - "Do model and brain have similar representational geometries?"
   - No alignment needed!

3. **PLS**: Predict brain activity from model
   - "Can we use the model to predict brain responses?"
   - Predictive modeling

Let's explore each!

## Part 2: Canonical Correlation Analysis (CCA)

### The Core Idea

CCA finds linear projections that maximize correlation between two datasets.

**Mathematical Setup:**

Given:
- Model representations: $X \in \mathbb{R}^{n \times p}$ (n stimuli, p model features)
- Brain responses: $Y \in \mathbb{R}^{n \times q}$ (n stimuli, q brain features)

Find:
- Projections $a, b$ such that $\text{corr}(Xa, Yb)$ is maximized

**Result:**
- Canonical correlations: $\rho_1 \geq \rho_2 \geq ... \geq \rho_k$
- Canonical variates: Transformed representations in shared space

### Implementing CCA

In [ ]:
# Generate synthetic "brain" and "model" data
n_stimuli = 100
n_model_features = 50
n_brain_features = 80

# Create some shared structure
shared_signal = np.random.randn(n_stimuli, 10)  # 10 shared dimensions

# Model representations (shared + specific)
model_specific = np.random.randn(n_stimuli, n_model_features - 10) * 0.5
model_data = np.concatenate([shared_signal, model_specific], axis=1)

# Brain representations (shared + specific + noise)
brain_specific = np.random.randn(n_stimuli, n_brain_features - 10) * 0.5
brain_noise = np.random.randn(n_stimuli, n_brain_features) * 0.3
brain_data = np.concatenate([shared_signal, brain_specific], axis=1) + brain_noise

print(f"Model data: {model_data.shape}")
print(f"Brain data: {brain_data.shape}")
print(f"Shared dimensions: 10 (ground truth)")

In [ ]:
# Perform CCA
n_components = 20

cca = SklearnCCA(n_components=n_components)
cca.fit(model_data, brain_data)

# Transform to canonical space
model_canonical, brain_canonical = cca.transform(model_data, brain_data)

# Compute canonical correlations
canonical_corrs = [np.corrcoef(model_canonical[:, i], brain_canonical[:, i])[0, 1] 
                   for i in range(n_components)]

print("\nCanonical Correlations:")
for i, corr in enumerate(canonical_corrs[:10]):
    print(f"  Component {i+1}: {corr:.3f}")

### Visualizing CCA Results

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top-left: Canonical correlations
axes[0, 0].bar(range(1, n_components+1), canonical_corrs)
axes[0, 0].axhline(0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')
axes[0, 0].set_xlabel('Canonical Component')
axes[0, 0].set_ylabel('Correlation')
axes[0, 0].set_title('Canonical Correlations')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Top-right: Scree plot
cumsum_corrs = np.cumsum(canonical_corrs) / np.sum(canonical_corrs)
axes[0, 1].plot(range(1, n_components+1), cumsum_corrs, 'o-', linewidth=2)
axes[0, 1].axhline(0.9, color='green', linestyle='--', alpha=0.5, label='90% threshold')
axes[0, 1].set_xlabel('Number of Components')
axes[0, 1].set_ylabel('Cumulative Sum of Correlations (normalized)')
axes[0, 1].set_title('Cumulative Canonical Correlations')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Bottom-left: First canonical component scatter
axes[1, 0].scatter(model_canonical[:, 0], brain_canonical[:, 0], alpha=0.6)
axes[1, 0].set_xlabel('Model Canonical Variable 1')
axes[1, 0].set_ylabel('Brain Canonical Variable 1')
axes[1, 0].set_title(f'First Canonical Component (r={canonical_corrs[0]:.3f})')
axes[1, 0].grid(True, alpha=0.3)

# Add regression line
z = np.polyfit(model_canonical[:, 0], brain_canonical[:, 0], 1)
p = np.poly1d(z)
axes[1, 0].plot(model_canonical[:, 0], p(model_canonical[:, 0]), 
                "r--", alpha=0.8, linewidth=2)

# Bottom-right: Second canonical component
axes[1, 1].scatter(model_canonical[:, 1], brain_canonical[:, 1], alpha=0.6, color='orange')
axes[1, 1].set_xlabel('Model Canonical Variable 2')
axes[1, 1].set_ylabel('Brain Canonical Variable 2')
axes[1, 1].set_title(f'Second Canonical Component (r={canonical_corrs[1]:.3f})')
axes[1, 1].grid(True, alpha=0.3)

z = np.polyfit(model_canonical[:, 1], brain_canonical[:, 1], 1)
p = np.poly1d(z)
axes[1, 1].plot(model_canonical[:, 1], p(model_canonical[:, 1]), 
                "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

print(f"\n✅ CCA successfully found {sum(np.array(canonical_corrs) > 0.5)} "
      f"strong canonical dimensions (r > 0.5)")

### Interpreting CCA Results

**Strong correlations (r > 0.7)**: Shared information between model and brain

**Moderate correlations (0.4 < r < 0.7)**: Partially shared representations

**Weak correlations (r < 0.4)**: Little shared structure

**Key insight**: The first few canonical components capture the most shared variance!

## Part 3: Representational Similarity Analysis (RSA)

### The Core Idea

RSA compares the **geometry** of representations without requiring alignment.

**Method**:
1. Compute Representational Dissimilarity Matrix (RDM) for model
2. Compute RDM for brain
3. Compare the two RDMs

**RDM**: Matrix where $RDM_{ij} = d(stimulus_i, stimulus_j)$

**Why RSA?**
- No alignment needed
- Scale invariant
- Captures second-order structure
- Works across different measurement types

### Computing RDMs

In [ ]:
def compute_rdm(data, metric='correlation'):
    """
    Compute Representational Dissimilarity Matrix.
    
    Args:
        data: (n_stimuli, n_features)
        metric: 'correlation', 'euclidean', 'cosine'
    
    Returns:
        RDM: (n_stimuli, n_stimuli)
    """
    if metric == 'correlation':
        # 1 - correlation = dissimilarity
        rdm = 1 - np.corrcoef(data)
    else:
        rdm = squareform(pdist(data, metric=metric))
    
    return rdm

# Compute RDMs
model_rdm = compute_rdm(model_data, metric='correlation')
brain_rdm = compute_rdm(brain_data, metric='correlation')

print(f"Model RDM: {model_rdm.shape}")
print(f"Brain RDM: {brain_rdm.shape}")

In [ ]:
# Visualize RDMs
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Model RDM
im1 = axes[0].imshow(model_rdm, cmap='viridis', aspect='auto')
axes[0].set_title('Model RDM')
axes[0].set_xlabel('Stimulus')
axes[0].set_ylabel('Stimulus')
plt.colorbar(im1, ax=axes[0], label='Dissimilarity')

# Brain RDM
im2 = axes[1].imshow(brain_rdm, cmap='viridis', aspect='auto')
axes[1].set_title('Brain RDM')
axes[1].set_xlabel('Stimulus')
axes[1].set_ylabel('Stimulus')
plt.colorbar(im2, ax=axes[1], label='Dissimilarity')

# Difference
diff = np.abs(model_rdm - brain_rdm)
im3 = axes[2].imshow(diff, cmap='RdYlGn_r', aspect='auto')
axes[2].set_title('Absolute Difference')
axes[2].set_xlabel('Stimulus')
axes[2].set_ylabel('Stimulus')
plt.colorbar(im3, ax=axes[2], label='|Model - Brain|')

plt.tight_layout()
plt.show()

### Comparing RDMs

In [ ]:
# Compare RDMs using correlation
# Extract upper triangular (excluding diagonal)
mask = np.triu(np.ones_like(model_rdm, dtype=bool), k=1)
model_rdm_vec = model_rdm[mask]
brain_rdm_vec = brain_rdm[mask]

# Compute Spearman correlation (rank-based, robust)
spearman_rho, spearman_p = spearmanr(model_rdm_vec, brain_rdm_vec)

# Compute Pearson correlation
pearson_r, pearson_p = pearsonr(model_rdm_vec, brain_rdm_vec)

print("RSA Similarity Scores:")
print(f"  Spearman ρ = {spearman_rho:.3f} (p = {spearman_p:.2e})")
print(f"  Pearson r  = {pearson_r:.3f} (p = {pearson_p:.2e})")

# Visualize scatter
plt.figure(figsize=(8, 8))
plt.scatter(model_rdm_vec, brain_rdm_vec, alpha=0.3, s=10)
plt.xlabel('Model Dissimilarity')
plt.ylabel('Brain Dissimilarity')
plt.title(f'RDM Comparison (Spearman ρ = {spearman_rho:.3f})')

# Add regression line
z = np.polyfit(model_rdm_vec, brain_rdm_vec, 1)
p = np.poly1d(z)
x_line = np.linspace(model_rdm_vec.min(), model_rdm_vec.max(), 100)
plt.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Interpretation
if spearman_rho > 0.5:
    print("\n✅ Strong representational similarity! Model and brain have similar geometry.")
elif spearman_rho > 0.3:
    print("\n⚠️  Moderate similarity. Some shared structure.")
else:
    print("\n❌ Weak similarity. Different representational geometries.")

## Part 4: Partial Least Squares (PLS)

### The Core Idea

PLS finds latent variables that maximize covariance and can predict brain activity from model.

**Advantages**:
- Handles multivariate output (many brain features)
- Finds latent structure
- Predictive (can test on new data)
- Works well with high-dimensional data

**Mathematical Setup**:

Find projections $t = Xw$ and $u = Yc$ such that $\text{cov}(t, u)$ is maximized.

### Implementing PLS

In [ ]:
# Split data into train/test
n_train = 80
train_idx = np.arange(n_train)
test_idx = np.arange(n_train, n_stimuli)

model_train = model_data[train_idx]
model_test = model_data[test_idx]
brain_train = brain_data[train_idx]
brain_test = brain_data[test_idx]

# Fit PLS
n_components_pls = 15
pls = PLSRegression(n_components=n_components_pls)
pls.fit(model_train, brain_train)

# Predict brain activity
brain_pred_train = pls.predict(model_train)
brain_pred_test = pls.predict(model_test)

# Compute R² scores
from sklearn.metrics import r2_score

r2_train = r2_score(brain_train, brain_pred_train)
r2_test = r2_score(brain_test, brain_pred_test)

print("PLS Prediction Performance:")
print(f"  Training R²: {r2_train:.3f}")
print(f"  Test R²:     {r2_test:.3f}")
print(f"\n  Using {n_components_pls} latent components")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Plot first 3 brain features
for i in range(3):
    # Training
    axes[0, i].scatter(brain_train[:, i], brain_pred_train[:, i], alpha=0.6)
    axes[0, i].plot([brain_train[:, i].min(), brain_train[:, i].max()],
                    [brain_train[:, i].min(), brain_train[:, i].max()],
                    'r--', linewidth=2)
    axes[0, i].set_xlabel(f'True Brain Feature {i+1}')
    axes[0, i].set_ylabel(f'Predicted')
    axes[0, i].set_title(f'Training (Feature {i+1})')
    axes[0, i].grid(True, alpha=0.3)
    
    # Test
    axes[1, i].scatter(brain_test[:, i], brain_pred_test[:, i], alpha=0.6, color='orange')
    axes[1, i].plot([brain_test[:, i].min(), brain_test[:, i].max()],
                    [brain_test[:, i].min(), brain_test[:, i].max()],
                    'r--', linewidth=2)
    axes[1, i].set_xlabel(f'True Brain Feature {i+1}')
    axes[1, i].set_ylabel(f'Predicted')
    axes[1, i].set_title(f'Test (Feature {i+1})')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 5: Statistical Evaluation

### Noise Ceiling

The **noise ceiling** estimates the maximum achievable performance given measurement noise.

**Why it matters**: If brain data has 30% noise, you can't achieve 100% prediction!

**Method**: Split-half reliability
1. Split brain recordings into two halves
2. Correlate the two halves
3. This gives upper bound on explainable variance

### Computing Noise Ceiling

In [ ]:
def estimate_noise_ceiling(data, n_splits=10):
    """
    Estimate noise ceiling via split-half correlation.
    
    Args:
        data: (n_stimuli, n_features) - with repetitions
        n_splits: number of random splits
    
    Returns:
        noise_ceiling: upper bound on explainable variance
    """
    correlations = []
    
    for _ in range(n_splits):
        # Random split
        perm = np.random.permutation(data.shape[0])
        half1 = data[perm[:len(perm)//2]]
        half2 = data[perm[len(perm)//2:]]
        
        # Average across features
        mean_corr = np.mean([np.corrcoef(half1[:, i], half2[:min(len(half1), len(half2)), i])[0, 1]
                            for i in range(min(half1.shape[1], half2.shape[1]))])
        correlations.append(mean_corr)
    
    # Spearman-Brown correction for full data
    mean_corr = np.mean(correlations)
    noise_ceiling = 2 * mean_corr / (1 + mean_corr)
    
    return noise_ceiling, correlations

# Estimate noise ceiling
noise_ceiling, split_corrs = estimate_noise_ceiling(brain_data, n_splits=20)

print(f"Noise Ceiling Estimate: {noise_ceiling:.3f}")
print(f"This means at most {noise_ceiling*100:.1f}% of variance is explainable")

# Normalize our scores
normalized_cca = canonical_corrs[0] / noise_ceiling
normalized_rsa = spearman_rho / noise_ceiling
normalized_pls = r2_test / noise_ceiling

print(f"\nNormalized Scores (fraction of explainable variance):")
print(f"  CCA: {normalized_cca:.2%}")
print(f"  RSA: {normalized_rsa:.2%}")
print(f"  PLS: {normalized_pls:.2%}")

### Confidence Intervals via Bootstrapping

In [ ]:
def bootstrap_ci(data1, data2, metric_fn, n_bootstrap=1000, ci=95):
    """
    Compute confidence interval via bootstrapping.
    
    Args:
        data1, data2: datasets to compare
        metric_fn: function(data1, data2) -> score
        n_bootstrap: number of bootstrap samples
        ci: confidence interval percentage
    
    Returns:
        (lower, upper, mean, std)
    """
    scores = []
    n = len(data1)
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        idx = np.random.choice(n, n, replace=True)
        score = metric_fn(data1[idx], data2[idx])
        scores.append(score)
    
    scores = np.array(scores)
    lower = np.percentile(scores, (100-ci)/2)
    upper = np.percentile(scores, 100 - (100-ci)/2)
    
    return lower, upper, np.mean(scores), np.std(scores)

# Bootstrap CCA
def cca_metric(X, Y):
    cca_temp = SklearnCCA(n_components=1)
    cca_temp.fit(X, Y)
    X_c, Y_c = cca_temp.transform(X, Y)
    return np.corrcoef(X_c[:, 0], Y_c[:, 0])[0, 1]

print("Computing bootstrap confidence intervals...")
lower, upper, mean, std = bootstrap_ci(model_data, brain_data, cca_metric, n_bootstrap=100)

print(f"\nCCA Score: {mean:.3f} ± {std:.3f}")
print(f"95% CI: [{lower:.3f}, {upper:.3f}]")

# Visualize
plt.figure(figsize=(10, 5))
plt.bar(['CCA', 'RSA (Spearman)', 'PLS (R²)'], 
        [canonical_corrs[0], spearman_rho, r2_test],
        yerr=[[canonical_corrs[0]-lower], [0.05], [0.08]],  # Approximate for demo
        capsize=10, edgecolor='black', alpha=0.7)
plt.ylabel('Score')
plt.title('Brain Alignment Scores with Confidence Intervals')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Part 6: Multi-Layer Alignment

### Finding Which Layers Match Brain Regions

Different model layers may align with different brain regions. Let's find the best matches!

In [ ]:
# Create a simple multi-layer model
class SimpleNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 30)
        self.layer2 = nn.Linear(30, 50)
        self.layer3 = nn.Linear(50, 40)
    
    def forward(self, x, return_all=False):
        h1 = torch.relu(self.layer1(x))
        h2 = torch.relu(self.layer2(h1))
        h3 = self.layer3(h2)
        
        if return_all:
            return {'layer1': h1, 'layer2': h2, 'layer3': h3}
        return h3

model = SimpleNetwork()

# Generate activations for all layers
inputs = torch.randn(n_stimuli, 10)
with torch.no_grad():
    layer_acts = model(inputs, return_all=True)

# Convert to numpy
layer_acts_np = {k: v.numpy() for k, v in layer_acts.items()}

print("Layer shapes:")
for layer, acts in layer_acts_np.items():
    print(f"  {layer}: {acts.shape}")

In [ ]:
# Compute alignment for each layer
layer_alignments = {}

for layer_name, acts in layer_acts_np.items():
    # CCA alignment
    cca_temp = SklearnCCA(n_components=min(10, acts.shape[1], brain_data.shape[1]))
    cca_temp.fit(acts, brain_data)
    X_c, Y_c = cca_temp.transform(acts, brain_data)
    cca_score = np.corrcoef(X_c[:, 0], Y_c[:, 0])[0, 1]
    
    # RSA alignment
    layer_rdm = compute_rdm(acts)
    mask = np.triu(np.ones_like(layer_rdm, dtype=bool), k=1)
    rsa_score, _ = spearmanr(layer_rdm[mask], brain_rdm[mask])
    
    layer_alignments[layer_name] = {
        'cca': cca_score,
        'rsa': rsa_score
    }

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

layers = list(layer_alignments.keys())
cca_scores = [layer_alignments[l]['cca'] for l in layers]
rsa_scores = [layer_alignments[l]['rsa'] for l in layers]

# CCA scores
axes[0].bar(layers, cca_scores, edgecolor='black')
axes[0].set_ylabel('CCA Score')
axes[0].set_title('CCA Alignment Across Layers')
axes[0].grid(True, alpha=0.3, axis='y')

# RSA scores
axes[1].bar(layers, rsa_scores, edgecolor='black', color='orange')
axes[1].set_ylabel('RSA Score (Spearman ρ)')
axes[1].set_title('RSA Alignment Across Layers')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Find best layer
best_cca_layer = max(layer_alignments.items(), key=lambda x: x[1]['cca'])[0]
best_rsa_layer = max(layer_alignments.items(), key=lambda x: x[1]['rsa'])[0]

print(f"\nBest aligned layers:")
print(f"  CCA: {best_cca_layer} (score: {layer_alignments[best_cca_layer]['cca']:.3f})")
print(f"  RSA: {best_rsa_layer} (score: {layer_alignments[best_rsa_layer]['rsa']:.3f})")

## Part 7: Complete Alignment Pipeline

Let's put it all together in a reusable pipeline:

In [ ]:
class BrainAlignmentPipeline:
    """
    Complete pipeline for model-to-brain alignment analysis.
    """
    
    def __init__(self, n_components=20):
        self.n_components = n_components
        self.results = {}
    
    def run_cca(self, model_data, brain_data):
        """Canonical Correlation Analysis"""
        cca = SklearnCCA(n_components=self.n_components)
        cca.fit(model_data, brain_data)
        X_c, Y_c = cca.transform(model_data, brain_data)
        
        corrs = [np.corrcoef(X_c[:, i], Y_c[:, i])[0, 1] 
                for i in range(self.n_components)]
        
        self.results['cca'] = {
            'correlations': corrs,
            'top_correlation': corrs[0],
            'mean_correlation': np.mean(corrs)
        }
    
    def run_rsa(self, model_data, brain_data):
        """Representational Similarity Analysis"""
        model_rdm = compute_rdm(model_data)
        brain_rdm = compute_rdm(brain_data)
        
        mask = np.triu(np.ones_like(model_rdm, dtype=bool), k=1)
        rho, p = spearmanr(model_rdm[mask], brain_rdm[mask])
        
        self.results['rsa'] = {
            'spearman_rho': rho,
            'p_value': p
        }
    
    def run_pls(self, model_data, brain_data, test_size=0.2):
        """Partial Least Squares"""
        n_train = int(len(model_data) * (1 - test_size))
        
        pls = PLSRegression(n_components=self.n_components)
        pls.fit(model_data[:n_train], brain_data[:n_train])
        
        pred_train = pls.predict(model_data[:n_train])
        pred_test = pls.predict(model_data[n_train:])
        
        r2_train = r2_score(brain_data[:n_train], pred_train)
        r2_test = r2_score(brain_data[n_train:], pred_test)
        
        self.results['pls'] = {
            'r2_train': r2_train,
            'r2_test': r2_test
        }
    
    def run_full_analysis(self, model_data, brain_data):
        """Run complete analysis pipeline"""
        print("Running complete brain alignment analysis...\n")
        
        print("1. Canonical Correlation Analysis...")
        self.run_cca(model_data, brain_data)
        
        print("2. Representational Similarity Analysis...")
        self.run_rsa(model_data, brain_data)
        
        print("3. Partial Least Squares...")
        self.run_pls(model_data, brain_data)
        
        self.print_report()
    
    def print_report(self):
        """Print formatted results"""
        print("\n" + "="*60)
        print("BRAIN ALIGNMENT REPORT")
        print("="*60)
        
        print("\n📊 CCA Results:")
        print(f"  Top correlation:  {self.results['cca']['top_correlation']:.3f}")
        print(f"  Mean correlation: {self.results['cca']['mean_correlation']:.3f}")
        
        print("\n📐 RSA Results:")
        print(f"  Spearman ρ: {self.results['rsa']['spearman_rho']:.3f}")
        print(f"  p-value:    {self.results['rsa']['p_value']:.2e}")
        
        print("\n🎯 PLS Results:")
        print(f"  Training R²: {self.results['pls']['r2_train']:.3f}")
        print(f"  Test R²:     {self.results['pls']['r2_test']:.3f}")
        
        print("\n" + "="*60)

# Run pipeline
pipeline = BrainAlignmentPipeline(n_components=15)
pipeline.run_full_analysis(model_data, brain_data)

## Summary & Next Steps

### What You've Mastered

1. ✓ **CCA** - Finding shared representational spaces via linear alignment
2. ✓ **RSA** - Comparing representational geometries without alignment
3. ✓ **PLS** - Predicting brain activity from model representations
4. ✓ **Statistical evaluation** - Noise ceilings, confidence intervals
5. ✓ **Multi-layer analysis** - Finding which layers match brain regions
6. ✓ **Complete pipelines** - End-to-end alignment workflows

### Key Takeaways

**When to use each method:**
- **CCA**: When you want linear alignment and care about correlations
- **RSA**: When you care about geometry and don't want to assume linearity
- **PLS**: When you want to predict brain activity from models

**Always remember:**
- Normalize by noise ceiling for fair comparison
- Use cross-validation for PLS
- Test statistical significance
- Compare multiple layers to find best matches

### Real-World Applications

You can now:
- Compare vision models to V1, V4, IT cortex
- Test if language models align with language areas
- Validate model architectures against brain data
- Predict neural responses to novel stimuli
- Guide model development toward biological plausibility

### Next Notebook

**[06_dynamical_systems.ipynb](06_dynamical_systems.ipynb)**

Learn to analyze neural trajectories using:
- Koopman operator theory
- Lyapunov exponents
- Fixed point analysis
- Controllability

### Further Reading

**Essential Papers**:
1. Kriegeskorte et al. (2008): "Representational Similarity Analysis"
2. Yamins & DiCarlo (2016): "Using goal-driven deep learning models to understand sensory cortex"
3. Schrimpf et al. (2020): "Brain-Score: Which Artificial Neural Network for Object Recognition is most Brain-Like?"
4. Nastase et al. (2019): "Measuring shared responses across subjects using intersubject correlation"

---

**Excellent work!** You can now rigorously compare artificial and biological neural networks! 🧠🤖